In [1]:
from edgeworth_env import *

2024-05-12 20:04:34,667	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-12 20:04:39,532	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-12 20:04:40,737	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
env = OligopolyMarket()

In [3]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

In [4]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [5]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-05-12 20:04:44,342	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.14
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [6]:
agent_ids = OligopolyMarket()._agent_ids
sym_policies = {agent_id: f"policy_firm_0" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = asym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)

config = (
        PPOConfig()
        .environment(OligopolyMarket)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 300000}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop,
                                 name="edgeworth_env",
                                 log_to_file=True,
                                 checkpoint_config = air.CheckpointConfig(
                                                        checkpoint_frequency=50,
                                                        checkpoint_at_end=True
                                                    )
                                 )
        )


output = tuner.fit()

2024-05-12 20:04:44,802	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-12 20:25:10,065	WARNING tune.py:194 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-05-12 20:25:11,144	INFO tune.py:1143 -- Total run time: 1226.50 seconds (1225.27 seconds for the tuning loop).
2024-05-12 20:25:11,146	WARNING tune.py:1158 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/Users/tristan/ray_results/edgeworth_env", trainab

(pid=2050) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=2050) 2024-05-12 20:04:46,461	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(PPO pid=2050) 2024-05-12 20:04:46,461	WARNING algorithm_config.py:672 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=2051) /opt/anaconda3/envs/RL/lib/python3.10/site-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=2051)   gym.logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=2051) 2024-05-12

In [ ]:
import IPython
%reload_ext tensorboard
%tensorboard --logdir ~/ray_results/edgeworth_env

In [ ]:
output.experiment_path

'/Users/tristan/ray_results/edgeworth_env'

In [ ]:
output._experiment_analysis.get_best_config()

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 0,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'num_learner_workers': 0,
 'num_gpus_per_learner_worker': 0,
 'num_cpus_per_learner_worker': 1,
 'local_gpu_idx': 0,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': True,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'torch_compile_learner': False,
 'torch_compile_learner_what_to_compile': <TorchCompileWhatToCompile.FORWARD_TRAIN: 'forward_train'>,
 'torch_compile_learner_dynamo_backend': 'aot_eager',
 'torch_compile_learner_dynamo_mode': None,
 'torch_compile_worker': Fa

In [ ]:
policies

{'firm_0': 'policy_firm_0',
 'firm_1': 'policy_firm_1',
 'firm_2': 'policy_firm_2',
 'firm_3': 'policy_firm_3',
 'firm_4': 'policy_firm_4'}

In [ ]:
for policy in policies.values():
    print(output.get_best_result().metrics["info"]["learner"][policy])

{'total_loss': 9.89549563328425, 'policy_loss': -0.04495230624452233, 'vf_loss': 9.93212248881658, 'vf_loss_unclipped': 137540396.15, 'vf_explained_var': 1.5386690696080525e-05, 'entropy': 1.7156886607408524, 'mean_kl_loss': 0.0185009660924455, 'default_optimizer_lr': 5.0000000000000016e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.45000001788139343}
{'total_loss': 9.938080807526907, 'policy_loss': -0.027383181142310302, 'vf_loss': 9.96098465124766, 'vf_loss_unclipped': 202438668.53333333, 'vf_explained_var': -1.4317532380421956e-06, 'entropy': 2.0656473408142726, 'mean_kl_loss': 0.009954095031207543, 'default_optimizer_lr': 5.0000000000000016e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.45000001788139343}
{'total_loss': 9.741447639465331, 'policy_loss': -0.031881986923205353, 'vf_loss': 9.768679253260295, 'vf_loss_unclipped': 32873433.016666666, 'vf_explained_var': 1.6206254561742146e-05, 'entropy': 2.169357344508171, 'mean_kl_loss': 0.01

In [ ]:
output.get_best_result().metrics["info"]["learner"].keys()

dict_keys(['__all__', 'policy_firm_3', 'policy_firm_0', 'policy_firm_2', 'policy_firm_4', 'policy_firm_1'])